<a href="https://colab.research.google.com/github/Excergic/Git_Tutorial/blob/main/Fine_Tuning_for_Acupuncture_Test_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.9 MB/s eta 0:00:00


In [ ]:
from datasets import Dataset

# Load your text file
with open("nano_acupuncture_data.txt", "r") as f:
    lines = f.readlines()

# Simple format: each line as a sample
data = {"text": [line.strip() for line in lines]}
dataset = Dataset.from_dict(data)

# Save as dataset
dataset.save_to_disk("acupuncture_dataset")

Saving the dataset (0/1 shards):   0%|          | 0/169 [00:00<?, ? examples/s]

In [ ]:
# Suppress API prompts
import os
os.environ["WANDB_MODE"] = "disabled"
os.environ["WANDB_DISABLED"] = "true"
os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "true"
os.environ["TRANSFORMERS_NO_ADVISORY_WARNINGS"] = "true"

from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments
from peft import LoraConfig, get_peft_model
from datasets import load_from_disk

# Load model and tokenizer
model_name = "deepseek-ai/deepseek-coder-1.3b-base"
model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

# Load dataset
dataset = load_from_disk("/content/acupuncture_dataset")

# Tokenize data with labels for causal LM
def tokenize_function(examples):
    encodings = tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)
    encodings["labels"] = encodings["input_ids"].copy()  # Labels are shifted input_ids
    return encodings

tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=["text"])  # Drop text column

# LoRA config
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none"
)
model = get_peft_model(model, lora_config)

# Training args (GPU-optimized)
training_args = TrainingArguments(
    output_dir="/content/lora_finetuned_acupuncture",
    per_device_train_batch_size=2,
    num_train_epochs=3,
    learning_rate=2e-4,
    logging_steps=10,
    save_steps=100,
    fp16=True,
    report_to="none"
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

# Fine-tune
trainer.train()

# Save
model.save_pretrained("/content/lora_finetuned_acupuncture")
tokenizer.save_pretrained("/content/lora_finetuned_acupuncture")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/631 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.69G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.69G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.37M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

Map:   0%|          | 0/169 [00:00<?, ? examples/s]

Step,Training Loss
10,2.377600
20,0.733100
30,0.600000
40,0.672300
50,0.569300
60,0.705400
70,0.645700
80,0.603500
90,0.389800
100,0.461100


('/content/lora_finetuned_acupuncture/tokenizer_config.json',
 '/content/lora_finetuned_acupuncture/special_tokens_map.json',
 '/content/lora_finetuned_acupuncture/tokenizer.json')